In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:461: FutureWarning: Passing (type, 1) or 

In [6]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)
# 运行次数
max_steps = 1000
# 图片数量
image_num = 3000
# 文件路径
DIR = '/Users/ricky/Desktop/TensorFlow-Learning/TensorFlow/'
# 定义会话
sess = tf.Session()
# 载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')
# 参数概要
def variable_summary(var):
    with tf.name_scope('summary'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
# 命名空间
with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,784],name='x_input')
    y = tf.placeholder(tf.float32,[None,10],name='y_input')
    
# 显示图片
with tf.name_scope('Input_reshape'):
    image_input_shape = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input_image',image_input_shape,10)
    
# 网路
with tf.name_scope('Layer_1'):
    with tf.name_scope('Weight'):
        W = tf.Variable(tf.truncated_normal([784,10]),name='w')
        variable_summary(W)
    with tf.name_scope('Bias'):
        b = tf.Variable(tf.zeros([10]) + 0.01,name='b')
        variable_summary(b)
    with tf.name_scope('WX_plus_b'):
        L1 = tf.matmul(x,W) + b
    with tf.name_scope('Softmax'):
        prediction = tf.nn.softmax(L1)
        
# loss
with tf.name_scope("Loss"):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction)
    tf.summary.scalar('loss',loss)
# train
with tf.name_scope("Train"):
    train_step = tf.train.AdadeltaOptimizer(0.1).minimize(loss)
    
# init
init = tf.global_variables_initializer()
sess.run(init)

with tf.name_scope("Accuracy"):
    # 相等为True，否则为False
    # tf.arg_max(y,1)，返回一维张量中最大值所在位置。
    # y , prediction是一个1维长度为10的张量
    correct_prediction = tf.equal(tf.arg_max(y,1),tf.arg_max(prediction,1))
    with tf.name_scope('accuracy'):
        # tf.cast()类型转换
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
# 产生metadata文件
if tf.gfile.Exists(DIR + 'projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR + 'projector/metadata.tsv')
with open(DIR + 'projector/metadata.tsv','w') as f:
    labels = sess.run(tf.argmax(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(labels[i]) + '\n')
        
# 合并summary
merged = tf.summary.merge_all()

projector_writer = tf.summary.FileWriter(DIR + 'projector/metadata.tsv',sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + 'projector/metadata.tsv'
embed.sprite.image_path = DIR + 'data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

for i in range(max_steps):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    summary, _ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},run_options=run_options,run_metadata=run_metadata)
    projector_writer.add_run_metadata(run_metadata,'step%03d' % i)
    projector_writer.add_summary(summary,i)
    
    if i%100 == 0:
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter:"+str(i) + ", Testing Accuracy:" + str(acc))
        
saver.save(sess,DIR + "projector/a_model.ckpt",global_step = max_steps)
projector_writer.close()
sess.close()   

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


FailedPreconditionError: /Users/ricky/Desktop/TensorFlow-Learning/TensorFlow/projector/metadata.tsv/projector_config.pbtxt